<a href="https://colab.research.google.com/github/hafluz/data-insights/blob/main/How_Green_is_the_Energy_We_Use_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **How Green is the Energy We Use?**
**A dashboard for tracking global low-carbon energy (i.e. hydro, nuclear, wind, solar) consumption and its impact on carbon emissions.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
from google.cloud import bigquery

project_id = "supple-walker-373820"
client = bigquery.Client(project=project_id)

In [ ]:
# # Renaming Column Names

# df = client.query('''

# ALTER TABLE `Energy.bp_review`
# RENAME COLUMN ISO3166_alpha3 TO country_code ;

# ALTER TABLE `Energy.worldbank_development_indicators`
# RENAME COLUMN GDP__current_US____NY_GDP_MKTP_CD_ TO GDP_current ;

# ALTER TABLE `Energy.worldbank_development_indicators`
# RENAME COLUMN Population__total__SP_POP_TOTL_ TO population ;

# ''' ).to_dataframe()

# print(df)


In [ ]:
# 1. What are the Biggest Energy Consuming Countries by Source?

df1 = client.query('''
SELECT country, year, primary_ej as total_cons, oilcons_ej as oil_cons, gascons_ej as natgas_cons, coalcons_ej as coal_cons, nuclear_ej as nuclear_cons, hydro_ej as hydro_cons, renewables_ej as renew_cons
FROM `Energy.bp_review` 
WHERE YEAR = 2021 
      AND country_code NOT LIKE 'T-%'
      AND country_code NOT LIKE 'O-%'
ORDER BY total_cons DESC
''' ).to_dataframe()

df1.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/biggest_energy_cons_1.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df1.to_csv(f)

,country,year,total_cons,oil_cons,natgas_cons,coal_cons,nuclear_cons,hydro_cons,renew_cons
0,Total World,2021,595.150850,184.213708,145.348586,160.103506,25.312823,40.259613,39.912613
1,China,2021,157.647225,30.598500,13.632999,86.169817,3.683567,12.246048,11.316294
2,US,2021,92.971659,35.326948,29.761828,10.571573,7.404259,2.427450,7.479601
3,India,2021,35.428369,9.406105,2.238042,20.088546,0.396993,1.510272,1.788411
4,Russian Federation,2021,31.297044,6.706621,17.086107,3.412108,2.010294,2.020912,0.061003
5,Japan,2021,17.740274,6.612353,3.730407,4.797545,0.553418,0.731336,1.315215
6,Canada,2021,13.936161,4.169663,4.290044,0.479886,0.831268,3.587586,0.577714
7,Germany,2021,12.641714,4.184094,3.259091,2.115600,0.623721,0.179923,2.279285
8,South Korea,2021,12.575524,5.390754,2.251719,3.036246,1.428368,0.028779,0.439657
9,Brazil,2021,12.565610,4.456353,1.456072,0.712471,0.132921,3.417763,2.390030


In [20]:
# 2. What are the most efficient Countries in the use of Oil to achieve the same unit of GDP? (US$ per barrel of oil)

df2 = client.query('''
WITH temp_table as (
SELECT a.country, a.year, a.primary_ej as total_cons, (a.oilcons_kbd*365) as oil_cons_per_year, CAST(b.GDP_current AS FLOAT64) as GDP_current
FROM `Energy.bp_review` a
JOIN `Energy.worldbank_development_indicators` b
    ON a.Year = b.Time
    AND a.country_code = b.Country_Code
WHERE b.GDP_current != '..' 
    AND region IS NOT NULL
ORDER BY total_cons DESC
)

SELECT country, oil_cons_per_year, GDP_current, ROUND(GDP_current/NULLIF(oil_cons_per_year,0)) as GDP_per_barrel
FROM temp_table
WHERE year = 2021
  AND oil_cons_per_year IS NOT NULL
ORDER BY GDP_per_barrel DESC ;
''' ).to_dataframe()

df2.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/gdp_per_bbl_2.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df2.to_csv(f)

,country,oil_cons_per_year,GDP_current,GDP_per_barrel
0,Switzerland,66055.948730,8.006402e+11,12120637.0
1,Ireland,48413.888752,5.041826e+11,10414008.0
2,Denmark,46766.393398,3.983033e+11,8516870.0
3,United Kingdom,451243.065050,3.131378e+12,6939448.0
4,Norway,72687.803418,4.821749e+11,6633504.0
5,Sweden,98420.369849,6.356638e+11,6458661.0
6,Israel,76324.866249,4.885265e+11,6400621.0
7,Bangladesh,65332.703639,4.162649e+11,6371464.0
8,Germany,746490.765700,4.259935e+12,5706614.0
9,France,519804.251140,2.957880e+12,5690372.0


In [5]:
# 3. What is the Share of Low-carbon energy in Global Power Consumption? (Per Country)

df3 = client.query('''
SELECT country, region, total_cons, nuclear_cons, hydro_cons, renew_cons, renew_total_cons, (renew_total_cons / total_cons) as share_of_renew
FROM (  SELECT country, region, year, primary_ej as total_cons, nuclear_ej as nuclear_cons, hydro_ej as hydro_cons, renewables_ej as renew_cons, (COALESCE(nuclear_ej,0) + COALESCE(hydro_ej,0) + COALESCE(renewables_ej,0)) as renew_total_cons
        FROM `Energy.bp_review` 
        WHERE year = 2021 
            AND country_code NOT LIKE 'T-%'
            AND country_code NOT LIKE 'O-%' )
ORDER BY share_of_renew DESC ;
''' ).to_dataframe()

df3.index = np.arange(1, len(df3) + 1)
df3.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/renew_share_3.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df3.to_csv(f)

,country,region,total_cons,nuclear_cons,hydro_cons,renew_cons,renew_total_cons,share_of_renew
1,Iceland,Europe,0.210187,NaN,0.130223,0.053149,0.183371,0.872420
2,Norway,Europe,2.045226,0.000000,1.347919,0.132131,1.480050,0.723661
3,Sweden,Europe,2.283887,0.479849,0.673141,0.489906,1.642896,0.719342
4,Finland,Europe,1.158715,0.215510,0.147961,0.253085,0.616556,0.532103
5,Switzerland,Europe,1.073902,0.167501,0.342476,0.058408,0.568384,0.529270
6,France,Europe,9.406049,3.429210,0.546047,0.739855,4.715112,0.501285
7,Brazil,S. & Cent. America,12.565610,0.132921,3.417763,2.390030,5.940714,0.472776
8,Denmark,Europe,0.671560,NaN,0.000154,0.274745,0.274899,0.409344
9,New Zealand,Asia Pacific,0.841422,NaN,0.228184,0.110465,0.338649,0.402472
10,Slovenia,Europe,0.269967,0.051579,0.044340,0.010675,0.106593,0.394838


In [14]:
# 4. What is the Total Energy Consumption Per Capita by country?

df4 = client.query('''
SELECT a.country, a.region, a.primary_ej as total_cons, CAST(b.population AS INT64) as population, COALESCE(a.primary_ej / CAST(b.population AS INT64)) as cons_per_capita
FROM `Energy.bp_review` a
LEFT JOIN `Energy.worldbank_development_indicators` b
    ON a.Year = b.Time
    AND a.country_code = b.Country_Code
WHERE year = 2021 
    AND region IS NOT NULL
ORDER BY cons_per_capita DESC ;
''' ).to_dataframe()

df4.index = np.arange(1, len(df4) + 1)
df4.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/cons_per_capita_4.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df4.to_csv(f)

,country,region,total_cons,population,cons_per_capita
1,Qatar,Middle East,1.927189,2688235,7.168977e-07
2,Singapore,Asia Pacific,3.456708,5453566,6.338436e-07
3,Iceland,Europe,0.210187,372520,5.642297e-07
4,United Arab Emirates,Middle East,4.528432,9365145,4.835411e-07
5,Kuwait,Middle East,1.738895,4250114,4.091407e-07
6,Trinidad & Tobago,S. & Cent. America,0.614571,1525663,4.028222e-07
7,Norway,Europe,2.045226,5408320,3.781630e-07
8,Canada,North America,13.936161,38246108,3.643812e-07
9,Oman,Middle East,1.499265,4520471,3.316612e-07
10,Saudi Arabia,Middle East,10.824213,35950396,3.010875e-07


In [15]:
# 5. What is the Evolution of Low-carbon Energy Consumption by Source since 1965?

df5 = client.query('''
SELECT country, year, primary_ej as total_cons, oilcons_ej as oil_cons, gascons_ej as natgas_cons, coalcons_ej as coal_cons, nuclear_ej as nuclear_cons, hydro_ej as hydro_cons, renewables_ej as renew_cons
FROM `Energy.bp_review` 
WHERE year BETWEEN 1965 AND 2021 
      AND country_code NOT LIKE 'T-%'
      AND country_code NOT LIKE 'O-%'
ORDER BY 1,2 ;
''' ).to_dataframe()

df5.head()

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/evo_cons_5.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df5.to_csv(f)

,country,year,total_cons,oil_cons,natgas_cons,coal_cons,nuclear_cons,hydro_cons,renew_cons
0,Algeria,1965,0.089398,0.055459,0.026750,0.002931,0.0,0.004258,NaN
1,Algeria,1966,0.107397,0.072982,0.027789,0.002847,0.0,0.003779,NaN
2,Algeria,1967,0.101691,0.068191,0.026958,0.002177,0.0,0.004365,NaN
3,Algeria,1968,0.109242,0.072602,0.028344,0.002303,0.0,0.005993,NaN
4,Algeria,1969,0.120741,0.076701,0.037266,0.002931,0.0,0.003843,NaN


In [ ]:
# 6. What is the Relationship between Energy Consumption and GDP per Capita?

df6 = client.query('''
SELECT a.country, a.region, CAST(b.population AS INT64) as population, a.primary_ej_pc as cons_per_capita, (CAST(b.GDP_current AS FLOAT64) / CAST(b.population AS INT64)) as GDP_per_capita
FROM `Energy.bp_review` a
LEFT JOIN `Energy.worldbank_development_indicators` b
    ON a.Year = b.Time
    AND a.country_code = b.Country_Code
WHERE year = 2021 
    AND region IS NOT NULL 
    AND b.GDP_current != '..' 
ORDER BY GDP_per_capita DESC ;
''' ).to_dataframe()

df6.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/cons_co2_pop_gdp_6.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df6.to_csv(f)

,country,region,year,population,cons_per_capita,GDP_per_capita
0,Luxembourg,Europe,2021,640064,234.729196,133590.146976
1,Ireland,Europe,2021,5033165,124.990336,100172.079253
2,Switzerland,Europe,2021,8703405,122.825674,91991.600458
3,Norway,Europe,2021,5408320,377.956471,89154.276093
4,Singapore,Asia Pacific,2021,5453566,630.277544,72794.003023
5,US,North America,2021,331893745,279.885479,70248.629000
6,Iceland,Europe,2021,372520,612.368317,68727.636665
7,Denmark,Europe,2021,5856733,114.927508,68007.756673
8,Qatar,Middle East,2021,2688235,686.211048,66838.357433
9,Sweden,Europe,2021,10415811,218.886837,61028.738060


In [16]:
# 7. What is the evolution of Low-carbon Energy Consumption (Renewables, Hydro, Nuclear) by country?

df7 = client.query('''
SELECT country, region, year, primary_ej as total_cons, nuclear_ej as nuclear_cons, hydro_ej as hydro_cons, renewables_ej as renew_cons, (COALESCE(nuclear_ej,0) + COALESCE(hydro_ej,0) + COALESCE(renewables_ej,0)) as renew_total_cons
FROM `Energy.bp_review` 
WHERE YEAR BETWEEN 2012 AND 2021 
      AND region IS NOT NULL
ORDER BY 1,2,3  ;
''' ).to_dataframe()

df7.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/renew_growth_7.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df7.to_csv(f)

,country,region,year,total_cons,nuclear_cons,hydro_cons,renew_cons,renew_total_cons
0,Algeria,Africa,2012,1.827619,NaN,0.003840,0.000264,0.004104
1,Algeria,Africa,2013,1.933117,NaN,0.000971,0.000281,0.001252
2,Algeria,Africa,2014,2.108208,NaN,0.001882,0.000595,0.002477
3,Algeria,Africa,2015,2.223065,NaN,0.001406,0.000746,0.002152
4,Algeria,Africa,2016,2.221185,NaN,0.000694,0.002545,0.003239
5,Algeria,Africa,2017,2.245661,NaN,0.000537,0.005011,0.005547
6,Algeria,Africa,2018,2.416901,NaN,0.001114,0.005839,0.006953
7,Algeria,Africa,2019,2.501585,NaN,0.001442,0.005931,0.007373
8,Algeria,Africa,2020,2.355889,NaN,0.000469,0.006549,0.007018
9,Algeria,Africa,2021,2.464503,NaN,0.000088,0.006354,0.006441


In [18]:
# 8. What is the Correlation between Renewable Energies use and CO2 Emissions per capita?

df8 = client.query('''
SELECT country, region, population, total_cons, cons_per_capita, renew_total_cons, (renew_total_cons / population) as renew_per_capita, co2_emissions_pc
FROM ( 
    SELECT a.country, a.region, CAST(b.population AS INT64) as population, a.primary_ej as total_cons,
          (COALESCE(nuclear_ej,0) + COALESCE(hydro_ej,0) + COALESCE(renewables_ej,0)) as renew_total_cons,  
          a.primary_ej_pc as cons_per_capita,
          a.co2_combust_pc as co2_emissions_pc
    FROM `Energy.bp_review` a
    LEFT JOIN `Energy.worldbank_development_indicators` b
        ON a.Year = b.Time
        AND a.country_code = b.Country_Code
    WHERE year = 2021 
        AND region IS NOT NULL 
      ) 
ORDER BY renew_per_capita DESC ;
''' ).to_dataframe()

df8.head(10)

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/cons_per_capita_8.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df8.to_csv(f)

,country,region,population,total_cons,cons_per_capita,renew_total_cons,renew_per_capita,co2_emissions_pc
0,Iceland,Europe,372520,0.210187,612.368317,0.183371,4.922452e-07,5.389011
1,Norway,Europe,5408320,2.045226,377.956471,1.480050,2.736616e-07,6.165293
2,Sweden,Europe,10415811,2.283887,218.886837,1.642896,1.577310e-07,3.842513
3,Canada,North America,38246108,13.936161,364.381047,4.996569,1.306425e-07,13.788780
4,Finland,Europe,5541017,1.158715,209.271242,0.616556,1.112712e-07,6.716765
5,France,Europe,67749632,9.406049,143.988003,4.715112,6.959612e-08,4.187533
6,New Zealand,Asia Pacific,5122600,0.841422,164.695506,0.338649,6.610880e-08,6.398642
7,Switzerland,Europe,8703405,1.073902,122.825674,0.568384,6.530597e-08,3.819017
8,Austria,Europe,8955797,1.482714,165.737764,0.555748,6.205454e-08,6.524121
9,Belgium,Europe,11592952,2.726125,235.775027,0.711719,6.139236e-08,9.924358


In [19]:
# 9. Which Countries Consume more Energy per Capita than World Average?

df9 = client.query('''
SELECT country, region, cons_per_capita, avg_per_capita, (cons_per_capita / avg_per_capita) as perc_above_avg
FROM (
    SELECT country, region, year, primary_ej_pc as cons_per_capita, AVG(primary_ej_pc) OVER() as avg_per_capita
    FROM `Energy.bp_review`
    WHERE year = 2021 
        AND region IS NOT NULL
    ) 
WHERE cons_per_capita > avg_per_capita    
ORDER BY cons_per_capita DESC ;
''' ).to_dataframe()

df9.index = np.arange(1, len(df9) + 1)
df9.head()

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/cons_per_capita_9.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df9.to_csv(f)

,country,region,cons_per_capita,avg_per_capita,perc_above_avg
1,Qatar,Middle East,686.211048,156.448676,4.386174
2,Singapore,Asia Pacific,630.277544,156.448676,4.028654
3,Iceland,Europe,612.368317,156.448676,3.914180
4,United Arab Emirates,Middle East,487.899821,156.448676,3.118593
5,Trinidad & Tobago,S. & Cent. America,438.312352,156.448676,2.801637


In [ ]:
# 10. What is the Share of Renewable Energy in Global Power Consumption?

df10 = client.query('''
SELECT country, total_cons, nuclear_cons, hydro_cons, renew_cons, renew_total_cons, (renew_total_cons / total_cons) as share_of_renew
FROM (  SELECT country, region, year, primary_ej as total_cons, nuclear_ej as nuclear_cons, hydro_ej as hydro_cons, renewables_ej as renew_cons, (COALESCE(nuclear_ej,0) + COALESCE(hydro_ej,0) + COALESCE(renewables_ej,0)) as renew_total_cons
        FROM `Energy.bp_review` 
        WHERE year = 2021 
           AND country = 'Total World' )
ORDER BY share_of_renew DESC ;
''' ).to_dataframe()

df10.head()

# Save As a CSV file To Google Drive:

# path = '/content/drive/My Drive/BigQuery_Download/renew_gloal_10.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df10.to_csv(f)

,country,total_cons,nuclear_cons,hydro_cons,renew_cons,renew_total_cons,share_of_renew
0,Total World,595.15085,25.312823,40.259613,39.912613,105.48505,0.177241


In [ ]:
# # Save As a CSV file To Google Drive

# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/output.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df.to_csv(f)

Mounted at /content/drive
